In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd

current_dir = Path(os.getcwd())
sys.path.append(str(current_dir.parent))  # 1階層上を追加

from metrics import get_result

In [2]:
res = pd.DataFrame()
for i in ["nci", "gdsc1", "gdsc2", "ctrp"]:
    true = pd.read_csv(f"true_{i}.csv", index_col=0)
    pred = pd.read_csv(f"pred_{i}.csv", index_col=0)
    res = pd.concat([res, get_result(true, pred, i)], axis=1)

In [3]:
res.T

,ACC,Precision,Recall,Specificity,F1,F2,Balanced ACC,G-Mean,AUROC,AUPR,LogLoss,Brier,MCC,Kappa,Youden J,Cost Ratio
NCI,0.780 (± 0.006),0.808 (± 0.006),0.734 (± 0.009),0.826 (± 0.006),0.769 (± 0.007),0.747 (± 0.008),0.780 (± 0.006),0.778 (± 0.006),0.864 (± 0.005),0.867 (± 0.005),0.465 (± 0.007),0.151 (± 0.003),0.562 (± 0.012),0.559 (± 0.012),0.729 (± 0.010),3.037 (± 0.121)
GDSC1,0.881 (± 0.002),0.970 (± 0.001),0.786 (± 0.006),0.975 (± 0.001),0.868 (± 0.003),0.817 (± 0.005),0.881 (± 0.002),0.876 (± 0.003),0.977 (± 0.001),0.978 (± 0.000),0.267 (± 0.005),0.083 (± 0.002),0.776 (± 0.004),0.762 (± 0.005),0.955 (± 0.001),4.538 (± 0.123)
GDSC2,0.878 (± 0.005),0.981 (± 0.001),0.771 (± 0.010),0.985 (± 0.001),0.864 (± 0.006),0.806 (± 0.009),0.878 (± 0.005),0.872 (± 0.006),0.984 (± 0.001),0.984 (± 0.001),0.261 (± 0.007),0.083 (± 0.003),0.774 (± 0.008),0.756 (± 0.010),0.968 (± 0.002),4.298 (± 0.184)
CTRP,0.863 (± 0.002),0.904 (± 0.002),0.813 (± 0.005),0.914 (± 0.003),0.856 (± 0.002),0.830 (± 0.004),0.863 (± 0.002),0.862 (± 0.002),0.944 (± 0.001),0.946 (± 0.001),0.318 (± 0.001),0.098 (± 0.000),0.730 (± 0.003),0.727 (± 0.003),0.888 (± 0.001),4.832 (± 0.112)
